In [19]:

list_of_packages <- c("ggplot2", "dplyr", "DESeq2","biomaRt","tidyr")
for(package in list_of_packages){
suppressPackageStartupMessages(suppressMessages(suppressWarnings(library(package,character.only=TRUE))))
}

In [36]:
data_path <- file.path("../data/gtex_data")
data_files <- list.files(data_path, pattern = "gct", full.names = TRUE)
rna_df <- data.frame()
for (file in data_files){
    tmp_df <- read.table(file, header = TRUE, skip = 2, sep = "\t", stringsAsFactors = FALSE)
    
        drop_cols <- c("Description")
        tmp_df <- tmp_df[, !names(tmp_df) %in% drop_cols]

        # make the df tidy long format
        tmp_df <- tmp_df %>% 
            gather(key = "Description", value = "expression", -1) %>% 
            mutate(tissue = gsub("../data/gtex_data/gene_reads_v10_", "", data_files[1])) %>%
            mutate(tissue = gsub(".gct", "", tissue))
    rna_df <- rbind(rna_df, tmp_df)
}
head(rna_df)
# tmp_df <- read.table(data_files[1], header = TRUE, skip = 2, sep = "\t", stringsAsFactors = FALSE)

: 

In [ ]:
# save the compiled gtex data to a file
all_gtex_tissue_data <- file.path("../data/all_gtex_tissue_data.tsv")
write.table(rna_df, file = all_gtex_tissue_data, sep = "\t", row.names = FALSE)